In [1]:
import pandas as pd
import numpy as np

df = pd.read_excel("RA.FRQ.xlsx", engine="calamine")

In [2]:
df.columns  

Index(['Loja', 'Cod. Loja', 'CNPJ', 'CLISAP', 'Grife', 'Artigo', 'Cor',
       'Tamanho', 'Sexo', 'Grupo', 'Linha', 'Vel. de Vendas', 'Lead Time',
       'Cobertura', 'Mínimo', 'Alvo', 'Em Loja', 'Em Transito', 'Em Carteira',
       'Necessidade', 'Ruptura', 'Ressuprir', 'Cancelada', 'Faltou no Estoque',
       'Atendido (SAP)', 'Valor', 'Nr. Pedido', 'Data Pedido', 'Data Embarque',
       'Data de Geração', 'Status Item', 'Status Pedido', 'Retorno Pedido',
       'Sem_Estoque', 'Franqueado'],
      dtype='object')

In [7]:
df['Retorno Pedido'] = df['Retorno Pedido'].str.split(',').str[0]

In [ ]:
filtro_cencelada = df['Cancelada'] > 0

TD_cancelado = df[filtro_cencelada]


TD_cancelado = TD_cancelado.groupby('Retorno Pedido').agg(
    total_registros=('Retorno Pedido', 'size'),
    total_cancelados=('Cancelada', 'sum'),
    total_lojas=('Loja', 'nunique'),
    total_franqueados=('Franqueado', 'nunique')

).reset_index()



,Retorno Pedido,total_registros,total_cancelados,total_lojas,total_franqueados
0,Ord./fornec.não é possível,186,448,3,2
1,SUSPENSO COBRANÇA,1383,4020,11,7


In [17]:
total_geral = TD_cancelado['total_registros'].sum()
TD_cancelado['%proporção'] = TD_cancelado['total_registros'] / total_geral

linha_total = {
    'Retorno Pedido': 'TOTAL',
    'total_registros': TD_cancelado['total_registros'].sum(),
    'total_cancelados': TD_cancelado['total_cancelados'].sum(),
    'total_lojas': TD_cancelado['total_lojas'].sum(),
    'total_franqueados': TD_cancelado['total_franqueados'].sum(),
    '%proporção': 1.0


}

df_total = pd.DataFrame([linha_total])
td_final = pd.concat([TD_cancelado, df_total], ignore_index=True)

def fmt_milhar(valor):
    return f"{valor:,.0f}".replace(',', ".")

def fmt_pct(valor):
    return f"{valor:.0%}"

colunas_numericas = ['total_registros', 'total_cancelados', 'total_lojas', 'total_franqueados']

for col in colunas_numericas:
    td_final[col] = td_final[col].apply(fmt_milhar)


td_final['%proporção'] = td_final['%proporção'].apply(fmt_pct)

td_final.head()

,Retorno Pedido,total_registros,total_cancelados,total_lojas,total_franqueados,%proporção
0,Ord./fornec.não é possível,186,448,3,2,12%
1,SUSPENSO COBRANÇA,1.383,4.020,11,7,88%
2,TOTAL,1.569,4.468,14,9,100%
